# **Tulbad:**


*   Kollased - välja võtta
*   Sinised - lahku lüüa "/"
*   Beež - küsitav







In [1]:
import numpy as np
import pandas as pd
from copy import *
import numbers
# TODO
data = pd.read_csv("clinvar_conflicting.csv")
data1 = deepcopy(data)

C:\Users\Ajangu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (0,38,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
#muudame nende tulpade sisu
data1[["Amino_acids initial", "Amino_acids replacement"]] = data1.Amino_acids.str.split("/",expand=True)
data1[["Codons initial", "Codons replacement"]] = data1.Codons.str.split("/",expand=True)

#eemaldame mittevajalikud tulbad
välja = ["Allele", "Amino_acids", "Codons", "CLNDN", "CLNDISDB", "CLNDISDBINCL", "CLNHGVS", "CLNVI", "MC", "SSR", "Feature_type", "Feature", "DISTANCE", "BAM_EDIT", "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE"]
for i in välja:
    data1 = data1.drop(labels=[i], axis=1)


#LIHTSASTI DEFINEERITAVAD TULBAD
inttüüpi = ["POS", "ORIGIN", "CLASS"]
floattüüpi = ["AF_ESP", "AF_EXAC", "AF_TGP", "LoFtool", "CADD_PHRED", "CADD_RAW" ,  "STRAND", "BLOSUM62" ]
strtüüpi = ["REF", "ALT", "Consequence", "IMPACT", "SYMBOL", "BIOTYPE" , "CLNVC", "IMPACT", "SIFT", "PolyPhen", 
            "Amino_acids initial", "Amino_acids replacement", "Codons initial", "Codons replacement", "cDNA_position" , 
            "CDS_position", "Protein_position"]


def datatypes(inttüüpi, strtüüpi, floattüüpi):
  for i in range(len(inttüüpi)):
      data1[inttüüpi[i]]=data1[inttüüpi[i]].astype(int)

  for i in range(len(strtüüpi)):
      data1[strtüüpi[i]]=data1[strtüüpi[i]].astype(str)
  
  for i in range(len(floattüüpi)):
      data1[floattüüpi[i]]=data1[floattüüpi[i]].astype(float)
      
datatypes(inttüüpi, strtüüpi, floattüüpi)
#ISIKLIKKU LÄHENEMIST VAJAVAD TULBAD
##CHROM
data1["CHROM"]=data1["CHROM"].astype(str)
data1.loc[data1["CHROM"] == "X", "CHROM"] = "23"
data1.loc[data1["CHROM"] == "MT", "CHROM"] = "24"


#CDNA, CDS ja protein tulpade töötlus
data1[["cDNA_pos_start" , "cDNA_pos_end"]] = data1.cDNA_position.str.split("-",expand=True)
data1[["CDS_pos_start" , "CDS_pos_end"]] = data1.CDS_position.str.split("-",expand=True)
data1[["Protein_pos_start" , "Protein_pos_end"]] = data1.Protein_position.str.split("-",expand=True)

data1.loc[data1["cDNA_pos_end"] == 'nan', "cDNA_pos_end"] = data1["cDNA_pos_start"]
data1.loc[data1["CDS_pos_end"] == 'nan', "CDS_pos_end"] = data1["CDS_pos_start"]
data1.loc[data1["Protein_pos_end"] == 'nan', "Protein_pos_end"] = data1["Protein_pos_start"]

data1.loc[data1["cDNA_position"].str.contains("?"), ["cDNA_position"]] = np.nan

pd.set_option('display.max_columns', None)
data1["EXON"]=data1["EXON"].astype(str)
data1["INTRON"]=data1["INTRON"].astype(str)

data1.loc[(data1["EXON"]!='nan') & (data1["INTRON"]!='nan') & (data1["REF"].str.len() == 1), ["INTRON", "EXON"]] = "TÕRGE"
data1.loc[data1["EXON"] == data1["INTRON"], ["INTRON", "EXON"]] = "TÕRGE"

error: nothing to repeat at position 0

In [ ]:
data1 = pd.get_dummies(data1,columns=list(data1.select_dtypes('object')), dummy_na=True)

In [ ]:
len(data1.columns)